In [1]:
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"    # 日志格式化输出
DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"                        # 日期格式
fp = logging.FileHandler('esio.log')
fs = logging.StreamHandler()
logging.basicConfig(level=logging.WARNING, format=LOG_FORMAT, datefmt=DATE_FORMAT, handlers=[fs])    # 调用

import pandas as pd
from pandasticsearch import DataFrame, Select
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, parallel_bulk
from tqdm import tqdm

from sklearn import datasets

from tool.sink import ESSink

In [2]:
df = datasets.load_iris()
df = pd.DataFrame(df.data, columns=['x1','x2','x3','x4'])
df.head(4)

,x1,x2,x3,x4
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2


In [3]:
es_localhost = "192.168.3.248:9200"
es_ysyp_test = "115.29.34.243:9200"
unsername = 'elastic'
password = 'superzsh123'

es = Elasticsearch(
    [es_ysyp_test],
    http_auth=(unsername, password)
)

In [4]:
%%time
user_feature = ESSink(df, es)
user_feature.to_es("user_feature")

CPU times: user 29.1 ms, sys: 8.11 ms, total: 37.2 ms
Wall time: 335 ms


In [5]:
%%time
# result_df = DataFrame.from_es(url='http://115.29.34.243:9200', username=unsername, password=password, index='index_bbs_browse_record', doc_type='type_bbs_browse_record', compat=5).to_pandas()
click_dict = es.search(index="index_bbs_browse_record", body={"query": {"match_all": {}}, "size":1000}, timeout='30m')
click_df = Select.from_dict(click_dict).to_pandas().drop(['_index', '_type', '_id', '_score'], axis=1)

pv_dict = es.search(index="index_syp_user_suggest", body={"query": {"match_all": {}}, "size":1000}, timeout='30m')
pv_df = Select.from_dict(pv_dict).to_pandas().drop(['_index', '_type', '_id', '_score'], axis=1)

CPU times: user 38.8 ms, sys: 9.01 ms, total: 47.8 ms
Wall time: 414 ms


In [6]:
click_df

,createtime,bbsprice,partitionId,maxValue,partitiontitle,brand_name,title,userid,parentid,brand_id,uid,times,minValue,parentidtitle,businessauthentication,bbsUserid,updatetime,bbsid,status,browseSource
0,2019-07-31 17:22:41,1.0,2017111154,3001.0,女表,,cartier 卡地亚手表,1038111778,2017111152,0,11447,1,0.0,,0,1038111617,2019-07-31 17:22:41,20172944,0.0,NaN
1,2019-08-05 11:08:14,66666.0,2017111190,1000000.0,斜挎包,,乔丹 大喊大叫大家觉得计算机等级,1038111556,2017111137,0,11869,1,46666.0,,0,1098119585,2019-08-05 11:08:14,20174327,0.0,NaN
2,2019-08-07 16:03:09,665.0,2017111190,3665.0,斜挎包,,爱彼 你就提吉里吉里经理,1038111556,2017111137,0,12034,1,0.0,,0,1038111610,2019-08-07 16:03:09,20173847,0.0,NaN
3,2019-08-08 19:08:57,600.0,2017111153,3600.0,男表,,雷达,1038111556,2017111152,0,12072,1,0.0,,0,1098119583,2019-08-08 19:08:57,20173649,0.0,NaN
4,2019-08-21 16:26:36,333.0,2017111146,3333.0,手包,,乔丹,1098119585,2017111137,0,12823,1,0.0,,0,1098119583,2019-08-21 16:26:36,20174763,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,2019-12-04 10:07:13,6123.0,2017111165,11123.0,围巾,,路易威登,1098119704,2017111162,0,20175736_1098119704,1,4123.0,,0,1038111611,2019-12-04 10:07:13,20175736,NaN,0.0
772,2019-12-03 19:26:34,3000.0,2017111148,6000.0,旅行箱,,ACNESTUDIOS新年新女性,1098119704,2017111137,0,20175783_1098119704,4,0.0,,1,1038111556,2019-12-04 10:18:54,20175783,NaN,0.0
773,2019-12-04 10:21:59,9999.0,2017111138,14999.0,单肩包,,CHANEL /香奈儿,1098119704,2017111137,0,20175769_1098119704,1,7999.0,,1,1098119582,2019-12-04 10:21:59,20175769,NaN,0.0
774,2019-12-02 17:51:09,10000.0,2017111138,15000.0,单肩包,,香奈儿Chanel女士单肩斜挎包流浪包小号,1098119704,2017111137,0,20175789_1098119704,2,8000.0,,1,1098119570,2019-12-04 10:22:05,20175789,NaN,0.0


In [7]:
pv_df.sort_values('createtime', ascending=0)

,userid,bbsid,batch,updatetime,createtime,bbs_createtime,userid_bbs,sug_type,isBusiAuth,useridentity,sug_version,sug_total
47,1098117798,20175820,3,2019-12-04 11:02:19,2019-12-04 11:02:19,2019-12-04 10:16:58,1098119693,3,0,1,0,0.0000
46,1098117798,20175819,4,2019-12-04 11:02:19,2019-12-04 11:02:19,2019-12-03 19:20:06,1098119704,3,1,2,0,0.0000
93,1098117798,20175815,2,2019-12-04 11:02:19,2019-12-04 11:02:19,2019-12-03 16:41:52,1098119552,3,0,1,0,0.0000
92,1098117798,20175808,9,2019-12-04 11:02:19,2019-12-04 11:02:19,2019-12-03 15:32:15,1098119554,3,0,1,0,0.0000
49,1098117798,20175812,3,2019-12-04 11:02:19,2019-12-04 11:02:19,2019-12-03 16:04:19,1098119552,3,0,1,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
64,1098119703,20175747,1,2019-11-29 20:59:01,2019-11-29 20:59:01,2019-11-28 19:53:33,1098119679,2,0,1,1,21.9465
9,1098119703,20175745,7,2019-11-29 20:59:01,2019-11-29 20:59:01,2019-11-28 19:50:50,1098119678,2,0,1,1,18.6185
63,1098119703,20175746,5,2019-11-29 20:59:01,2019-11-29 20:59:01,2019-11-28 19:52:04,1098119678,2,0,1,1,19.1305
62,1098119703,20175736,9,2019-11-29 20:59:01,2019-11-29 20:59:01,2019-11-28 19:16:51,1038111611,2,0,1,1,18.3145


In [8]:
es.search?